# Learned Sensing Matrix

We got an encoder-decoder like system set up using a similar architecture with DeepCodec. Now let's grab the sensing matrix, load it into MATLAB, and see if we can get better results using a learned sensing matrix on MNIST!

In [5]:
import torch
from torch import nn

In [6]:
class WFUMNISTCodec(nn.Module):
    """A NN that definitely won't work, lmao, Test our dataloader."""
    def __init__(self, original_res, sensing_res):
        """Makes a FCN type architecture, takes in picture dimension d x d"""
        super(WFUMNISTCodec, self).__init__()
        
        self.m = sensing_res
        # Sensing matrix
        self.sensing = torch.nn.Linear(original_res * original_res, sensing_res * sensing_res)
        
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 8, kernel_size=4, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(4, 1, kernel_size=3, stride=1, padding=1)
        self.upsample = torch.nn.UpsamplingBilinear2d(size=(original_res, original_res))
        
        
        
    def forward(self, x):
        # Apply the sensing matrix without activation, reshape back to image-like for FCN
        x = self.sensing(x.view(x.shape[0], -1)).view(x.shape[0], 1, self.m, self.m)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        return self.upsample(x)

In [7]:
wfunet_28_to_10 = torch.load('wfunet_conv_28_to_10.torchmodel')

## Getting the Sensing Matrix

The sensing matrix is just the first fully connected layer that maps MNIST into a lower dimensional space. Let's return this and serialize it into something that MATLAB can load.

In [11]:
params = list()]

[Parameter containing:
 tensor([[-0.0347,  0.0294, -0.0240,  ..., -0.0275, -0.0327,  0.0181],
         [-0.0134,  0.0102, -0.0059,  ...,  0.0121,  0.0262,  0.0328],
         [ 0.0167, -0.0179,  0.0238,  ..., -0.0242,  0.0053, -0.0210],
         ...,
         [-0.0215,  0.0185,  0.0147,  ..., -0.0204, -0.0232, -0.0202],
         [ 0.0302, -0.0287,  0.0326,  ...,  0.0324,  0.0339, -0.0128],
         [ 0.0015,  0.0131, -0.0350,  ..., -0.0029, -0.0168, -0.0316]],
        device='cuda:0', requires_grad=True), Parameter containing:
 tensor([-0.0266, -0.0339, -0.0299,  0.0167,  0.0380, -0.0219, -0.0200, -0.0265,
          0.0149, -0.0076,  0.0079, -0.0102,  0.0221, -0.0294,  0.0256,  0.0408,
         -0.0012,  0.0360,  0.0366, -0.0115,  0.0353, -0.0265, -0.0273,  0.0024,
         -0.0470, -0.0365,  0.0101,  0.0154, -0.0330, -0.0013, -0.0255, -0.0278,
         -0.0027, -0.0272,  0.0051,  0.0030, -0.0567, -0.0143,  0.0116, -0.0102,
          0.0182,  0.0091, -0.0276, -0.0069, -0.0598, -0.0308, 

In [12]:
params = list(wfunet_28_to_10.parameters())

In [15]:
params[0]

AttributeError: 'Parameter' object has no attribute 'value'

In [17]:
params[0].detach().numpy()

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [18]:
params[0].detach().cpu().numpy()

array([[-0.03465402,  0.02938243, -0.02398746, ..., -0.02754745,
        -0.03266645,  0.01814092],
       [-0.0134211 ,  0.01023765, -0.00588348, ...,  0.01208362,
         0.02619076,  0.03276626],
       [ 0.01674922, -0.01788615,  0.02383197, ..., -0.02418784,
         0.00529132, -0.0209921 ],
       ...,
       [-0.02151089,  0.01851771,  0.01473175, ..., -0.02037549,
        -0.0231528 , -0.02024649],
       [ 0.03021049, -0.02869857,  0.0325641 , ...,  0.03239095,
         0.03393048, -0.01284097],
       [ 0.00145506,  0.0130661 , -0.03503685, ..., -0.00294045,
        -0.01682661, -0.03155801]], dtype=float32)

In [19]:
params[0].detach().cpu().numpy().shape

(100, 784)

In [20]:
a = params[0].detach().cpu().numpy()

In [21]:
from scipy.io import savemat

In [23]:
savemat('sensing_mnist_100_784.mat', {'A': a})